In [1]:
from util import *
from DataStructure import *
import networkx as nx
try:
    import pydot
    from networkx.drawing.nx_pydot import write_dot
except ImportError:
    raise

In [2]:
class parser_util():
    def __init__(self):
        pass
    
    def check_computation(self):
        self.must_parse("main")
        #more than 0
        while self.tokens[0].type in ["var","array"]:
            self.check_varDecl()
        #more than 0
        while self.tokens[0].type in ["function","procedure"]:
            self.check_funcDecl()
        bb=self.new_bb([])
        self.must_parse("OPEN_C")        
        bb=self.check_stat_Sequence(bb)
        self.must_parse("CLOSE_C")
        self.must_parse("POINT")
        
    def check_funcDecl(self):
        if self.tokens[0].type=="function":
            self.must_parse("function")
        else:
            self.must_parse("procedure")
        
        bb=self.new_bb([])

        self.must_parse("INDENT")
        self.check_formalParam()
        self.must_parse("SEMI")
        self.check_funcBody(bb)
        self.must_parse("SEMI")
        
    def check_formalParam(self):
        if self.tokens[0].type=="OPEN_P":
            self.must_parse("OPEN_P")
            if self.tokens[0].type=="INDENT":
                self.must_parse("INDENT")
                #more than 0
                while self.tokens[0].type=="COMMA":            
                    self.must_parse("COMMA")
                    self.must_parse("INDENT")
            self.must_parse("CLOSE_P")
         
    def check_funcBody(self,bb):
        while self.tokens[0].type in ["var","array"]:
            self.check_varDecl()
        self.must_parse("OPEN_C")        
        bb=self.check_stat_Sequence(bb)
        self.must_parse("CLOSE_C")

    def check_relation(self,bb):
        result_x,_=self.check_expression(bb)
        op=self.tokens[0].value
        self.check_relOp()
        result_y,_=self.check_expression(bb)
        self.compute_SSA(result_x,result_y,"CMP",bb)
        return op
        
    def check_relOp(self):
        if self.tokens[0].type in ['OP_EQ','OP_NEQ','OP_LESS','OP_LEQ','OP_GRT','OP_GEREQ']:
            self.must_parse(self.tokens[0].type)
        else:
            print("error in cheching relOp!",self.tokens[0].type)
        
    def must_parse(self,target_token):        
        sym=self.tokens.popleft()
        if (sym.type==target_token):
            if sym.type=="INDENT":
                print(sym.value),
            pass
        else:
            print("error!!!!!",target_token,sym,inspect.stack()[1][3])
            print([i.value for i in self.tokens])         
    
    def check_returnStatement(self,bb):
        self.must_parse("return")
        #how?
        if self.tokens[0].type in ["INDENT","NUMBER","OPEN_P","call"]:
            self.check_expression(bb)
    
    def check_designator(self,bb):
        indent=self.tokens[0].real_value
        self.must_parse("INDENT")        
        while self.tokens[0].type=="OPEN_B":
            self.must_parse("OPEN_B")
            self.check_expression(bb)
            self.must_parse("CLOSE_B")
        return indent
    
    def check_term(self,bb):
        copy_propagation=True
        result_x,copy_propagation=self.check_factor(bb)
        while self.tokens[0].value in ["*","/"]:
            copy_propagation=False
            op=self.tokens[0].value
            self.must_parse("OP")
            result_y,_=self.check_factor(bb)
            result_x=self.compute_SSA(result_x,result_y,op,bb)
            if result_x.kind!="constant":
                result_x.real_value=self.Random_variable.get_current_random_variable()                
                self.add_SSA_table(result_x.real_value,bb)
        self.Random_variable.next()
        return result_x,copy_propagation

    def check_funcCall(self):
        bb=0
        self.must_parse("call")
        self.must_parse("INDENT")
        if self.tokens[0].type=="OPEN_P":
            self.must_parse("OPEN_P")
            #how?
            if self.tokens[0].type in ["INDENT","NUMBER","OPEN_P","call"]:
                self.check_expression(bb)
                while self.tokens[0].type=="COMMA":
                        self.must_parse("COMMA")
                        self.check_expression(bb)
            self.must_parse("CLOSE_P")
        
    def check_varDecl(self):
        self.check_typeDecl()
        self.must_parse("INDENT")
        #more than 0 var
        while self.tokens[0].type in ["COMMA"]:
            self.must_parse("COMMA")            
            self.must_parse("INDENT")
        self.must_parse("SEMI")
        
    def check_typeDecl(self):
        if self.tokens[0].type=="array":
            self.must_parse("array")
            counter=0
            #more than 1
            while self.tokens[0].type=="OPEN_B":
                self.must_parse("OPEN_B")
                self.must_parse("NUMBER")
                self.must_parse("CLOSE_B")
                counter+=1
            if counter==0:
                print("error typeDecl",self.tokens[0].type)
                
        elif self.tokens[0].type=="var":
            self.must_parse("var")
        else:
            print("error typeDecl",target_token,self.tokens[0].type)
        
    def check_stat_Sequence(self,bb):
        bb=self.check_statement(bb)
        # more that 0
        while self.tokens[0].type in ["SEMI"]:
            self.must_parse("SEMI")            
            bb=self.check_statement(bb)
        return bb
            
    def check_statement(self,bb):
        if self.tokens[0].type not in ["let","call","if","while","return"]:
            print("error check_statement",self.tokens[0].type,inspect.stack()[1][3])
        if self.tokens[0].type=="let":
            self.check_assignment(bb)
        if self.tokens[0].type=="call":
            self.check_funcCall()
        if self.tokens[0].type=="if":
            bb=self.check_ifStatement(bb)
        if self.tokens[0].type=="while":
            bb=self.check_whileStatement(bb)
        if self.tokens[0].type=="return":
            self.check_returnStatement(bb)
        return bb
        
    def check_factor(self,bb):
        result=Result(None,None,None,None,None)
        copy_propagation=True
        if self.tokens[0].type not in ["INDENT","NUMBER","OPEN_P","call"]:
            print("error,check_factor!!!",self.tokens[0],inspect.stack()[1][3])
        if self.tokens[0].type=="INDENT":
            #TODO:what about array?
            result.kind,result.address,result.real_value='var',self.tokens[0].value,self.tokens[0].real_value
            indent=self.check_designator(bb)          
        if self.tokens[0].type=="NUMBER":
            copy_propagation=True
            result.kind,result.val='constant',self.tokens[0].value 
            self.must_parse("NUMBER")            
        if self.tokens[0].type=="OPEN_P":
            copy_propagation=False
            self.must_parse("OPEN_P")
            result=self.check_expression(bb)
            self.must_parse("CLOSE_P")
        if self.tokens[0].type=="call":
            copy_propagation=False
            self.check_funcCall()
        return result,copy_propagation
            
    
    def check_expression(self,bb):
        result_x,copy_propagation=self.check_term(bb)
        #can be optimized by adding all constant at first!
        while self.tokens[0].value in ["+","-"]:
            copy_propagation=False
            op=self.tokens[0].value
            self.must_parse("OP")
            result_y,_=self.check_term(bb)
            result_x=self.compute_SSA(result_x,result_y,op,bb)
            if result_x.kind!="constant":
                result_x.real_value=self.Random_variable.get_current_random_variable()                
                self.add_SSA_table(result_x.real_value,bb)
        self.Random_variable.next()
        return result_x,copy_propagation
        
    def check_assignment(self,bb):
        self.must_parse("let")
        indent=self.check_designator(bb)
        self.must_parse("ASSIGN")
        result_x,copy_propagation=self.check_expression(bb)
        
        self.ASSIGNMENT_SSA_builder(copy_propagation,result_x,indent,bb)

    def _remove_loops(self,G_reverse,reverse=False):
        try:
            reverse_edges=nx.find_cycle(G_reverse, orientation='original')
            while len(reverse_edges)>0:
                reverse_edges=nx.find_cycle(G_reverse, orientation='original')
                print(reverse_edges)
                for e in reverse_edges:
                    if not reverse:
                        if e[0]<e[1]:
                            G_reverse.remove_edge(e[0],e[1])
                    if reverse:
                        if e[0]>e[1]:
                            G_reverse.remove_edge(e[0],e[1])
        except:
            print("no loop found!")
        return G_reverse
    
    def _Phi_inst_table_extract(self,Phi_inst):
        result=dict()
        for inst in Phi_inst:
            try:
                result[inst.x]= Parenthesize(int(inst.SSA_number))
            except:
                pass
        return result     
            
    def _SSA_while(self,join_node,outside_while_bb):        
        #I need to copy the SSA of inside_while_bb into join node, and update the SSA_code based on the updated table
        #need a heavy refactorization!
        self.copy_SSA_table(join_node)
        
        #need to move phi to the top:bring all the phi to the top of join_node by traversing the whole list
        aa=self.node[join_node]["code"]
        self.node[join_node]["code"]=[]
        Phi_inst=[i for i in aa if i.op=="Phi"]
        not_Phi_inst=[i for i in aa if i.op!="Phi"]
        
        Phi_inst_replace_table=self._Phi_inst_table_extract(Phi_inst)
        #CHECK: need to update the SSA code: is this correct?
        # update join_node code based on the Phi
        output_code=self._replace_while_phi(target=join_node,Phi_table=Phi_inst_replace_table,input_code=not_Phi_inst)
        self.node[join_node]["code"]=copy.deepcopy(Phi_inst+output_code)
        
        self.build_graph()#self.G
        G_wo_loop=self._remove_loops(self.G,reverse=True)
        while_child=nx.descendants(G_wo_loop,join_node)
        #print("=========================while loop update",while_child)
        for i in while_child:
            output_code=self._replace_while_phi(target=i,Phi_table=Phi_inst_replace_table,input_code=self.node[i]["code"])
            self.node[i]["code"]=copy.deepcopy(output_code)
        
        #why did I save the SSA numbers () in string format? what is the alternative?        
        def find_SSA_number(i,bb):
            pass
                
        #I need to add the jmp the the join block as well!
        # RelOp jump from the end of join_node to the outside_while_bb
        #self.compute_SSA(outside_while_bb,None,op,join_node)
        
    def _replace_while_phi(self,target,Phi_table,input_code):
        for i in input_code:
            i.x= Phi_table[i.x] if i.x in Phi_table else i.x
            i.y= Phi_table[i.y] if i.y in Phi_table else i.y                
        return input_code 
    
    def check_ifStatement(self,bb):
        mother_bb=bb
        
        self.must_parse("if")
        op=self.check_relation(bb)
        
        self.must_parse("then")    
        newbbif1=self.new_bb([mother_bb])        
        newbbif=self.check_stat_Sequence(newbbif1)
        
        if self.tokens[0].type=="else":
            self.must_parse("else")
            newbbelse1=self.new_bb([mother_bb])
            newbbelse=self.check_stat_Sequence(newbbelse1)
            merge_bb=self.new_bb([newbbif,newbbelse])
            self._make_bra_IF_ELSE(True,merge_bb,newbbif,newbbelse,mother_bb,op)
            self.if_head.append((merge_bb,newbbif,newbbelse))
            self.compute_SSA(newbbelse1,None,op,mother_bb)
        else:
            merge_bb=self.new_bb([newbbif,mother_bb])
            self._make_bra_IF_ELSE(False,merge_bb,newbbif,None,mother_bb,op)
            self.if_head.append((merge_bb,newbbif,mother_bb))        
            self.compute_SSA(merge_bb,None,op,mother_bb)
        
        self.must_parse("fi")                
        return merge_bb
    
    #bra at the end of mother block(mother_bb), jumping to the beginning else/merge block(newbbelse/merge_bb)!
    def _make_bra_IF_ELSE(self,has_else,merge_bb,newbbif,newbbelse,mother_bb,op):
        if has_else:#bra at the end of if block(newbbif), jumping to the beginning of merge block(merge_bb)!
            self.compute_SSA(merge_bb,None,"BRA",newbbif)        #(x,y,op,bb)
            #self.compute_SSA(newbbelse,None,op,mother_bb)
        else:#conditional branch at the end of mother_BB block, to the beginning of merge block(merge_bb)!
            #self.compute_SSA(merge_bb,None,op,mother_bb)
            pass
            

    def check_whileStatement(self,bb):
        join_mother=bb
        join_node=self.new_bb([bb])
        
        self.must_parse("while")
        op=self.check_relation(join_node)
        
        self.must_parse("do")
        inside_while_bb=self.new_bb([join_node])
        inside_while_bb_end=self.check_stat_Sequence(inside_while_bb)
        
        self.node[join_node]["parents"].append(inside_while_bb_end)#what about dominator?
        
        self.must_parse("od")        
        
        #self._SSA_while should come before outside_while_bb=self.new_bb!!
        self._SSA_while(join_node,None)
        outside_while_bb=self.new_bb([join_node])
          
        self.compute_SSA(outside_while_bb,None,op,join_node)

        # will be used later in the register allocation
        self.loop_head.append((join_node,inside_while_bb,join_mother))
        
        self.compute_SSA(join_node,None,"BRA",inside_while_bb_end)#jump at the end of inside_while to the join block
        
        return outside_while_bb

    def remove_cse_inst(self,ii):
        dom_bb,bb,dom_comp,jj=ii
        #if dom_comp.op in ["CMP"] or dom_comp.op in opRel.values():return
        #self.SSA_number=SSA_number
        jj.op="CSE"
        jj.x=Parenthesize(jj.SSA_number)+"CSE"
        jj.y=Parenthesize(dom_comp.SSA_number)+"CSE"
        
        replace_with=Parenthesize(dom_comp.SSA_number)
        replace=Parenthesize(jj.SSA_number)
        
        for i in self.node:
            for j in self.node[i]["code"]:
                if j.x==replace:
                    j.x=replace_with
                if j.y==replace:
                    j.y=replace_with
            
        #update SSA_table
        for i in self.node:
            for k,v in self.node[i]["SSA_table"].items():
                for cnt,vv in enumerate(v):
                    if vv==jj.SSA_number:
                        v[cnt]=dom_comp.SSA_number
        #self.cse_replace()

In [3]:
class SSA_util():
    def __init__(self):
        pass
        
    def add_SSA_current_line(self):
        self.SSA_current_line+=1
        return self.SSA_current_line
    
    def get_SSA_current_line(self):
        return self.SSA_current_line       
    
    def add_SSA_table(self,x,bb):
        self.node[bb]["SSA_table"][x].append(self.get_SSA_current_line())
    
    def _add_SSA_table_wo_init(self,x,bb):
        self.node[bb]["SSA_table"][x].append(-1)
        
    def get_SSA_table_variable(self,x,bb):
        if x in self.node[bb]["SSA_table"].keys():
            pass
        else:#what about variables without initilization?
            self._add_SSA_table_wo_init(x,bb)
        return self.node[bb]["SSA_table"][x][-1]
    
    def __repr__(self):
        return self.SSA_variable_table       

In [6]:
class SSA(SSA_util):
    def __init__(self):
        self.SSA_current_line=0
        SSA_util.__init__(self)
    
    def emit2(self,bb,op,x=None,y=None):
        self.add_SSA_current_line()
        code_line=SSA_code(self.get_SSA_current_line(),op,x,y)
        self.node[bb]["code"].append(code_line)
    
    def compute_SSA(self,x,y,op,bb):
        if op=="MOVE":
            self.emit2(bb,op,Parenthesize(self.get_SSA_table_variable(x,bb)),Parenthesize(self.get_SSA_table_variable(y.real_value,bb)))
        if op in opCode.keys():
            if x.kind=='constant' and y==None: #let a<-1;
                self.emit2(bb,"None","#")
            elif x.kind=='var' and y==None: #let a<-1;
                self.emit2(bb,"None?",x.real_value)
            elif x.kind=='constant' and y.kind=='constant':
                x.val= eval(str(x.val) + op + str(y.val))
                #dont know the correct OP
                self.emit2(bb,"ADDI","#"+str(x.val))
            else:
                if x.kind=='constant':
                    x,y=y,x
                    self.emit2(bb,opCode[op],Parenthesize(self.get_SSA_table_variable(x.real_value,bb)),"#"+str(y.val))
                elif y.kind=='constant':
                    self.emit2(bb,opCode[op],Parenthesize(self.get_SSA_table_variable(x.real_value,bb)),"#"+str(y.val))
                else:
                    self.emit2(bb,opCode[op],Parenthesize(self.get_SSA_table_variable(x.real_value,bb)),Parenthesize(self.get_SSA_table_variable(y.real_value,bb)))
            #print("compute_SSA",x)
            return x
        elif op in ["CMP","BRA"] or op in opRel.keys():
            self._compute_SSA_branch(x,y,op,bb)
        else:
            print("!!!! could not find the operation:",op)
            
    def _compute_SSA_branch(self,x,y,op,bb):
        if op in ["CMP"]:
            if x.kind=='constant' and y.kind=='constant':
                #can be optimized to avoid either if or else BB!!
                #x.val= eval(str(x.val) + op + str(y.val))
                self.emit2(bb,"CMP","#"+str(x.val),"#"+str(y.val))
            else:
                if x.kind=='constant':
                    self.emit2(bb,"CMP","#"+str(x.val),str(self.get_SSA_table_variable(y.real_value,bb)))
                elif y.kind=='constant':
                    self.emit2(bb,"CMP",Parenthesize(self.get_SSA_table_variable(x.real_value,bb)),"#"+str(y.val))
                else:
                    self.emit2(bb,"CMP",Parenthesize(self.get_SSA_table_variable(x.real_value,bb)),Parenthesize(self.get_SSA_table_variable(y.real_value,bb)))
                
        elif op in opRel.keys():
            jump_add=self._get_BB_first_SSA_number(x)
            print(bb,opRel[op],Parenthesize(jump_add))
            self.emit2(bb,opRel[op],"A"+Parenthesize(jump_add))
        elif op in ["BRA"]:
            #jump address is the first SSA number of merge block (passed as X)
            jump_add=self._get_BB_first_SSA_number(x)
            self.emit2(bb,"BRA","A"+Parenthesize(jump_add))
                
    def insert_phi(self,bb,var,left_line,right_line):
        #print("inserting phi...")
        self.emit2(bb,'Phi',Parenthesize(left_line),Parenthesize(right_line))
        self.add_SSA_table(var,bb)   
        
    def _set_copy_propagation_SSA_table(self,x,y,bb):
        # variable x is copying y: x<-y
        if y.kind!="constant":
            try:
                self.node[bb]["SSA_table"][x].append(self.node[bb]["SSA_table"][y.real_value][-1])
            except:#there is no SSA entry for y: maybe I have initilize it to 0!
                self.node[bb]["SSA_table"][x].append(-1)
        else:
            self.node[bb]["SSA_table"][x].append(self.get_SSA_current_line())
             
    def copy_SSA_table(self,bb): #copy the parent's SSA table
        for p in self.node[bb]["parents"]:
            for k,v in self.node[p]["SSA_table"].items():
                if TEMP_VARIABLE_CORE in k: continue
                if k not in self.node[bb]["SSA_table"].keys():
                    self.node[bb]["SSA_table"][k]=copy.deepcopy(v)
                    
                if len(self.node[bb]["parents"])==1:continue #we Dont need phi for just one parent.
                elif self.node[bb]["SSA_table"][k]==v:continue #the variable SSA has not changed
                else:#insert Phi
                    self.insert_phi(bb,k,self.node[bb]["SSA_table"][k][-1],self.node[p]["SSA_table"][k][-1]) 
    
    def ASSIGNMENT_SSA_builder(self,copy_propagation,result,indent,bb):
        if copy_propagation and result.kind!="constant": #x<-y
            self.emit2(bb,"#copy propagation",indent,result.real_value)
            self._set_copy_propagation_SSA_table(indent,result,bb)
        elif result.kind=="constant": #x<-2
            self.emit2(bb,"ADDI",indent,result.val)
            self._set_copy_propagation_SSA_table(indent,result,bb)
        else: #x <- y + z
            self.add_SSA_table(indent,bb)  
    
    def check_CSE_two_bb(self,dom_bb,bb):
        result=[]
        for dom_comp in self.node[dom_bb]["code"]:
            if dom_comp.op=="BRA" or dom_comp.op in opRel.values():continue
            for j in self.node[bb]["code"]: 
                if dom_comp==j and dom_comp.SSA_number!=j.SSA_number and dom_comp.op!="CSE" and dom_comp.op!="None":
                    print(dom_bb,bb,dom_comp,j,"are the sameeeee: CSE")
                    self.remove_cse_inst((dom_bb,bb,dom_comp,j))
        return result
        
    def build_graph(self):
        self.G=nx.DiGraph()
        for i in self.node.keys():
            self.G.add_node(i, label=str(i))
        for i in self.node.keys():
            for j in self.node[i]["parents"]:
                self.G.add_edge(j,i)
       
    def Remove_CSE(self):
        dom_list=[]
        for entry_n in self.entry_node:
            d=collections.defaultdict(list)            
            dom_list+= [(y,x) for x,y in nx.immediate_dominators(self.G, entry_n).items()]
        G_dominator=nx.DiGraph(dom_list)
        for i in G_dominator.nodes():
            cc=nx.descendants(G_dominator,source=i)
            cc.add(i)
            #print("Dom treee_reachable from node:",i,cc)
            for domed in cc:
                cse_result=self.check_CSE_two_bb(i,domed)

In [5]:
class BB(SSA_code):
    def __init__(self):
        self.node=defaultdict(lambda: defaultdict(list))
        self.number=0
        self.G=None
        self.entry_node=[1]
        
    def _get_BB_first_SSA_number(self,bb):
        first_line="where?"
        try:
            first_line=self.node[bb]["code"][0].SSA_number
        except:
            try: #in the case there is no SSA code in the target BB, we add an empty line to it
                self.emit2(bb,"None","For the sake of IF")
                first_line=self.node[bb]["code"][0].SSA_number
            except:
                print("!!! could not find the first line of block:",bb)                
        return first_line
    
    def new_bb(self,parents):
        self.number=self.number+1
        self.node[self.number]["parents"]=parents
        self.node[self.number]["code"]=[]
        self.node[self.number]["code_reg_alloc"]=[]
        self.node[self.number]["live_in"]=set()
        self.node[self.number]["live_out"]=set()
        self.node[self.number]["SSA_table"]=defaultdict(list)
        self.node[self.number]["live_in"]=set()
        self.node[self.number]["live_out"]=set()
        self.node[self.number]["live_out_left"]=set()
        self.node[self.number]["live_out_right"]=set()
        self.copy_SSA_table(self.number)
        return self.number

    def _vis(self,G,show,v="0"):
        custom_node_attrs = {}
        for i in self.node.keys():
            cc="\n".join([str(j) for j in self.node[i][show]])            
            G.add_node(str(i)+v, label=str(i)+"\n"+cc)
            custom_node_attrs[str(i)+v] = str(i)+cc
        for i in self.node.keys():
            for j in self.node[i]["parents"]:
                G.add_edge(str(j)+v,str(i)+v)
        return G
    
    def visulize(self,show="code"):
        G=nx.DiGraph()
        if show=="code":
            G=self._vis(G,show)
        
            custom_node_attrs = {}
            for i in self.node.keys():
                aa=[[k]+v for k,v in self.node[i]["SSA_table"].items() if TEMP_VARIABLE_CORE not in k]
                cc="\n".join([ str(jj) for jj in aa])
                G.add_node(str(i)+"1", label="SSA_table:"+str(i)+"\n"+cc)
                custom_node_attrs[str(i)+"1"] = str(i)+cc
            for i in self.node.keys():
                for j in self.node[i]["parents"]:
                    G.add_edge(str(j)+"1",str(i)+"1")
        elif show=="code_reg_alloc":
            G=self._vis(G,show="code",v="0")
            G=self._vis(G,show="code_reg_alloc",v="1")        
        
        write_dot(G, "grid.dot")
        
    def visulize_live(self,v,G=None):        
        if G==None:G=nx.DiGraph()
            
        custom_node_attrs = {}
        
        for i in self.node.keys():
            s_in="in:"+str( sorted(self.node[i]["live_in"]) )
            if len(self.node[i]["live_out_left"])>0:
                s_out="out_left:"+str( sorted(self.node[i]["live_out_left"]) )
                s_out+="\nout_right:"+str( sorted(self.node[i]["live_out_right"]) )
            else:
                s_out="out:"+str( sorted(self.node[i]["live_out"]) )
            cc="\n".join([s_out,s_in])
            
            G.add_node(str(i)+v, label="live:"+str(i)+"\n"+cc)
            custom_node_attrs[str(i)+v] = str(i)+cc
            
        for i in self.node.keys():
            for j in self.node[i]["parents"]:
                G.add_edge(str(j)+v,str(i)+v)
        write_dot(G, "grid_live.dot")
        return G
    
    def print_bb(self):
        print("^^^^ printing BBs:")
        for i in self.node.keys():
            print("====BB=====",i)
            for j in self.node[i]["code"]:
                print(j)
            print("SSA_table=",self.node[i]["SSA_table"])
        for i in self.node.keys():
            print("bb",i,"connects to bb:",self.node[i]["parents"])   